In [10]:
from tagger.core.mongo.models.order import Order, OrderCodeMap
from datetime import datetime
from tagger.models import Inventory
from tagger.models.inventory import InventoryStatus, ProductType
from tagger.core.label.receiving_label import make_receiving_label
from tagger.core.label.print_label import print_label
from bson import ObjectId
from tagger.core.mongo.models.product import Product
from tagger.core.mongo.models.alloff_product import AlloffProduct

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
orders = Order.objects(orderstatus__nin=[
    "CREATED", "RECREATED", "PAYMENT_PENDING",
    "CANCEL_FINISHED", "DELIVERY_FINISHED", "CONFIRM_PAYMENT", "DELIVERY_STARTED"
])
order_map = {}

for order in orders:
    if order.orderstatus not in order_map:
        order_map[order.orderstatus] = []
    order_map[order.orderstatus].append(order)

In [3]:
def make_inventory(order: Order, is_return=False, target_item_index=None):
    if len(order.orders) > 1 and (target_item_index is None or target_item_index >= len(order.orders)):
        raise "Multiple items in the order... specify correct item index!"
    
    item = order.orders[0 if len(order.orders) == 1 else target_item_index]
    product = item.product if item.product is not None else item.alloffproduct
    product_type = ProductType.NORMAL_PRODUCT if item.product is not None else ProductType.TIMEDEAL_PRODUCT
    
    today = datetime.now().date()
    
    today_inventory_count = Inventory.objects.filter(created__gte=today).count()
    code_prefix = today.isoformat()[2:].replace("-", "")
    current_sequence = today_inventory_count + 1  # 0 inventories --> current is #1 (starts with 1)
    code = f"{code_prefix}-{str(current_sequence).zfill(3)}"
    if is_return:
        code += "-R"
    
    
    
    return Inventory.objects.create(
        code=code,
        status=InventoryStatus.IN_STOCK,
        size=item.size,
        product_id=str(product._id),
        product_from_order_url=f"https://office.alloff.co/orders/{order.id}",
        product_name=product.name,
        product_brand=product.brand.keyname,
        product_type=product_type
    )
    

In [14]:
inventory = make_inventory(Order.get_by_code("m36bc"))
print_label(make_receiving_label(inventory))

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1358: RuntimeWarning: DateTimeField Inventory.created received a naive datetime (2021-11-08 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "


In [24]:
is_return=False
inventory = make_inventory(Order.objects(id=ObjectId("616bfc4c17fc2e5a1f9e8d24")).first(), is_return=is_return)
print_label(make_receiving_label(inventory))

In [12]:
for i in Inventory.objects.all():
    product = (
        Product.objects(id=ObjectId(i.product_id)) if i.product_type == ProductType.NORMAL_PRODUCT 
        else AlloffProduct.objects(id=ObjectId(i.product_id))).first()
    
    i.product_brand = product.brand.keyname
    i.save()
    
               